In [ ]:
import random
from itertools import combinations
from collections import defaultdict

# Apriori algorithm to find frequent itemsets
def apriori(D, min_support):
    item_counts = defaultdict(int)
    for transaction in D:
        for item in transaction:
            item_counts[frozenset([item])] += 1

    frequent_itemsets = {itemset: count for itemset, count in item_counts.items() if count >= min_support}

    k = 2
    current_itemsets = list(frequent_itemsets.keys())

    while current_itemsets:
        candidate_itemsets = defaultdict(int)

        for transaction in D:
            transaction_set = frozenset(transaction)
            for itemset in combinations(current_itemsets, k):
                union_itemset = set().union(*itemset)
                if union_itemset.issubset(transaction_set):
                    candidate_itemsets[frozenset(union_itemset)] += 1

        frequent_k_itemsets = {itemset: count for itemset, count in candidate_itemsets.items() if count >= min_support}
        frequent_itemsets.update(frequent_k_itemsets)

        current_itemsets = list(frequent_k_itemsets.keys())
        k += 1

    return frequent_itemsets

# Fitness function calculation
def calculate_fitness(particle, w1, w2, w3, FI, S, NS):
    alpha = calculate_fth(particle, FI, S)
    beta  = calculate_nth(particle, FI, NS)
    gamma = calculate_ntg(particle)
    return w1 * alpha + w2 * beta + w3 * gamma

# Calculate alpha (FI - S)
def calculate_fth(particle, FI, S):
    return len(FI) - len(S)

# Calculate beta (FI - NS)
def calculate_nth(particle, FI, NS):
    return len(FI) - len(NS)

# Placeholder for gamma (e.g., a complexity measure)
def calculate_ntg(particle):
    return 0

# Velocity update for particles
def update_velocity(pbest, gbest, particle, et, n1, n2, n3, n4):
    new_velocity = set()
    if pbest - particle:
        new_velocity.update(random.sample(pbest - particle, min(n1, len(pbest - particle))))
    if gbest - particle:
        new_velocity.update(random.sample(gbest - particle, min(n2, len(gbest - particle))))
    if et:
        new_velocity.update(random.sample(et, min(n3, len(et))))
    return new_velocity

# Position update for particles
def update_position(particle, velocity, n4):
    new_position = particle.union(velocity)
    if particle:
        new_position.update(random.sample(particle, min(n4, len(particle))))
    return new_position

# Identify victim items
def find_victim_items(S, FI):
    NS = [frozenset(itemset) for itemset, _ in FI if frozenset(itemset) not in S]
    VI = set()
    remaining_S = S.copy()
    while remaining_S:
        item_counts = {}
        for itemset in remaining_S:
            for item in itemset:
                if item in item_counts:
                    item_counts[item] += 1
                else:
                    item_counts[item] = 1
        max_count = max(item_counts.values())
        candidates = [item for item, count in item_counts.items() if count == max_count]
        if len(candidates) > 1:
            min_ns_occurrence = float('inf')
            victim_item = None
            for candidate in candidates:
                ns_occurrence = sum(1 for itemset in NS if candidate in itemset)
                if ns_occurrence < min_ns_occurrence:
                    min_ns_occurrence = ns_occurrence
                    victim_item = candidate
        else:
            victim_item = candidates[0]
        VI.add(victim_item)
        remaining_S = [itemset for itemset in remaining_S if victim_item not in itemset]
    return VI

# Calculate mj
def calculate_mj(vi, S, FI, delta, n, D):
    max_support = max([count for itemset, count in FI if vi in itemset and frozenset(itemset) in S], default=0)
    mj = n * ((max_support - (delta * len(D))) + 1)
    return max(int(mj), 1)

# VIDPSO algorithm
def vidpso_algorithm(D, S, delta, M):

    min_support = int(delta * len(D))

    FI = apriori(D, min_support)
    print("Frequent Itemsets (FI):", FI)

    FI_list = [(list(itemset), count) for itemset, count in FI.items()]

    # Calculate NS as FI - S
    NS = [frozenset(itemset) for itemset, _ in FI_list if frozenset(itemset) not in S]
    print("NS:", NS)

    VI = find_victim_items(S, FI_list)
    print("Victim Items:", VI)

    VTi = {vi: [TID for TID, transaction in enumerate(D) if vi in transaction] for vi in VI}

    particles = []
    n = 1  # Adjusting coefficient, can be tuned based on the dataset
    for _ in range(M):
        particle = []
        for vi in VI:
            sample_size = calculate_mj(vi, S, FI_list, delta, n, D)
            particle.append(set(random.sample(VTi[vi], min(sample_size, len(VTi[vi])))))
        particles.append(particle)
    print("Initial Particles:", particles)

    pbest = particles[:]
    fitness_values = [calculate_fitness(particle, 0.8, 0.1, 0.1, FI_list, S, NS) for particle in particles]
    gbest = particles[fitness_values.index(min(fitness_values))]
    print("Initial Global Best:", gbest)
    print("Initial Fitness Values:", fitness_values)
    print("Initial Global Best Fitness:", calculate_fitness(gbest, 0.8, 0.1, 0.1, FI_list, S, NS))

    iterations = 3


    for iter_num in range(iterations):
        print(f"\nIteration {iter_num + 1}:")
        for j, vi in enumerate(VI):
            ETj = set(VTi[vi])
            for particle in particles:
                ETj -= particle[j]

            for i, particle in enumerate(particles):
                n1 = int(0.3 * len(pbest[i][j])) if 0.3 * len(pbest[i][j]) <= abs(len(pbest[i][j]) - len(particles[i][j])) else abs(len(pbest[i][j]) - len(particles[i][j]))
                n2 = int(0.3 * len(gbest[j])) if 0.3 * len(gbest[j]) <= abs(len(gbest[j]) - len(particles[i][j])) else abs(len(gbest[j]) - len(particles[i][j]))
                n3 = int(0.3 * len(ETj)) if 0.3 * len(ETj) <= len(ETj) else len(ETj)
                n4 = len(particles[i][j]) - n1 - n2 - n3

                velocity = update_velocity(pbest[i][j], gbest[j], particle[j], ETj, n1, n2, n3, n4)
                particles[i][j] = update_position(particle[j], velocity, n4)

                print(f"Particle {i+1}, Victim Item {vi}:")
                print(f"  n1 = {n1}, n2 = {n2}, n3 = {n3}, n4 = {n4}")
                print(f"  Velocity = {velocity}")
                print(f"  Position (after update) = {particles[i][j]}")

            fitness_values = [calculate_fitness(particle, 0.8, 0.1, 0.1, FI_list, S, NS) for particle in particles]
            for i, fitness_value in enumerate(fitness_values):
                if fitness_value > calculate_fitness(pbest[i], 0.8, 0.1, 0.1, FI_list, S, NS):
                    pbest[i] = particles[i]
                if fitness_value > calculate_fitness(gbest, 0.8, 0.1, 0.1, FI_list, S, NS):
                    gbest = particles[i]

                print(f"  Particle {i+1} Fitness Value: {fitness_value}")

        # Modify just one transaction ID based on the best particle
        sanitized_dataset = D[:]
        modified_tids = set()
        for j, vi in enumerate(VI):
            # Pick just one TID from the best global position to modify
            if gbest[j]:
                TID_to_modify = random.choice(list(gbest[j]))
                sanitized_dataset[TID_to_modify] = [item for item in sanitized_dataset[TID_to_modify] if item != vi]
                modified_tids.add(TID_to_modify)
                break  # Stop after modifying one transaction ID

        print("Sanitized Dataset (after one modification):", sanitized_dataset)
        print("Modified TIDs:", modified_tids)

    return sanitized_dataset, gbest, modified_tids


# Example dataset
D = [
    ['a', 'b', 'e'],
    ['b', 'c', 'e'],
    ['a', 'b', 'c'],
    ['a', 'c', 'e', 'd'],
    ['b', 'c', 'e'],
    ['b', 'c', 'e'],
    ['b', 'c', 'd', 'e'],
    ['a', 'b', 'e'],
    ['a', 'b'],
    ['c', 'e']
]

S = [{'a', 'b'}, {'b', 'c'},{'c', 'e'}]

delta = 0.4
M = 5

# Run the algorithm
sanitized_dataset = vidpso_algorithm(D, S, delta, M)
print("Sanitized Dataset:", sanitized_dataset)


Frequent Itemsets (FI): {frozenset({'a'}): 5, frozenset({'b'}): 8, frozenset({'e'}): 8, frozenset({'c'}): 7, frozenset({'b', 'a'}): 4, frozenset({'b', 'e'}): 6, frozenset({'b', 'c'}): 5, frozenset({'e', 'c'}): 6, frozenset({'b', 'c', 'e'}): 4}
NS: [frozenset({'a'}), frozenset({'b'}), frozenset({'e'}), frozenset({'c'}), frozenset({'b', 'e'}), frozenset({'e', 'b', 'c'})]
Victim Items: {'a', 'c'}
Initial Particles: [[{0}, {9, 4, 5}], [{7}, {3, 4, 5}], [{2}, {1, 3, 9}], [{8}, {9, 3, 5}], [{0}, {1, 4, 6}]]
Initial Global Best: [{0}, {9, 4, 5}]
Initial Fitness Values: [5.1000000000000005, 5.1000000000000005, 5.1000000000000005, 5.1000000000000005, 5.1000000000000005]
Initial Global Best Fitness: 5.1000000000000005

Iteration 1:
Particle 1, Victim Item a:
  n1 = 0, n2 = 0, n3 = 0, n4 = 1
  Velocity = set()
  Position (after update) = {0}
Particle 2, Victim Item a:
  n1 = 0, n2 = 0, n3 = 0, n4 = 1
  Velocity = set()
  Position (after update) = {7}
Particle 3, Victim Item a:
  n1 = 0, n2 = 0, n

<ipython-input-8-b63815a6fe39>:62: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  new_velocity.update(random.sample(et, min(n3, len(et))))
<ipython-input-8-b63815a6fe39>:69: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  new_position.update(random.sample(particle, min(n4, len(particle))))
<ipython-input-8-b63815a6fe39>:60: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  new_velocity.update(random.sample(gbest - particle, min(n2, len(gbest - particle))))
